# Cable Bridge

In [ ]:
''' Inputs '''
tower_base = 6                   # Metres below the origin
tower_height_to_first_stay = 10  # Metres above the origin
cable_spacing_tower = 1          # Metre increments

cable_spacing_forwards = [5,5,5,5,5,5,5,5]
cable_spacing_backwards = [5,5,5,5,5,5,5,5]

deck_width = 5.0
deck_depth = 0.6

tower_width_base = 2.0
tower_depth_base = 2.6
tower_width_top = 2.0
tower_depth_top = 2.6

stay_dia = 0.15

In [ ]:
import numpy as np
import math
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.22.0") # Connect to LUSAS Modeller

# If there is not currently a model open, create one
if not lusas.existsDatabase():
    lusas.newProject("Structural", "Cable Bridge")
else:
    import sys; sys.path.append('../') # Locate the helpers file
    from m100_Tools_And_Helpers import Helpers
    Helpers.initialise(lusas)
    Helpers.reset_database_contents(lusas.database())

db = lusas.database() # Reference to the model database for convenience

# 2D model with Y vertical
db.setAnalysisCategory("2D")
db.setVerticalDir("Y")

# Set the unit system
db.setModelUnits("kN,m,t,s,C")

In [ ]:
''' Create Model Points '''

point_origin = Helpers.create_point(0,0,0)
tower_points = [Helpers.create_point(0,-tower_base,0)]
y = tower_height_to_first_stay
for i in range(0, len(cable_spacing_forwards)):
    tower_points.append(Helpers.create_point(0,y,0))
    y+=cable_spacing_tower


deck_points_forwards = [Helpers.create_point(x,0,0) for x in np.sum.accumulate(cable_spacing_forwards)]
deck_points_backwards = [Helpers.create_point(-x,0,0) for x in np.sum.accumulate(cable_spacing_backwards)]
deck_points = deck_points_backwards.reverse() + deck_points_forwards


In [ ]:
''' Create model lines '''

tower_lines = []
for i in range(0, len(tower_points)-1):
    tower_lines.append(Helpers.create_line(tower_points[i], tower_points[i+1]))

deck_lines = []
for i in range(0, len(deck_points)-1):
    deck_lines.append(Helpers.create_line(deck_points[i], deck_points[i+1])) 

back_stays = []
for i in range(0, len(deck_points_backwards)-1):
    back_stays.append(Helpers.create_line(deck_points_backwards[i], tower_points[i+2])) 

front_stays = []
for i in range(0, len(deck_points_forwards)-1):
    front_stays.append(Helpers.create_line(deck_points_forwards[i], tower_points[i+2])) 

In [ ]:
''' Create Geometric Sections '''
deck_section_attr = Helpers.create_rectangular_section(db, "Deck", deck_width, deck_depth)
tower_section_attr = Helpers.create_rectangular_section(db, "Tower", tower_width_base, tower_depth_base)
stay_section_attr = Helpers.create_circular_section(db, "Stay", stay_dia)

deck_section_attr.assignTo(deck_lines)
tower_section_attr.assignTo(tower_lines)
stay_section_attr.assignTo(back_stays)
stay_section_attr.assignTo(front_stays)

In [ ]:
''' Create Material Attributes'''

# Steel Material attribute
steel_material = db.createIsotropicMaterial("Steel", 200_000, 0.3, 7.8e-9, 10e-6)

# Concrete Material attribute
concrete_material = db.createIsotropicMaterial("Concrete", 30_000, 0.2, 2.4e-9, 10e-6)

steel_material.assignTo(back_stays)
steel_material.assignTo(front_stays)
concrete_material.assignTo(deck_lines)
concrete_material.assignTo(tower_lines)

In [ ]:
''' Support attributes ''' 
fixed = db.createSupportStructural("Fixed in XY").setStructural("R", "R", "F", "F", "F", "F", "F", "F", "C", "F")
slide = db.createSupportStructural("Fixed in Y").setStructural("F", "R", "F", "F", "F", "F", "F", "F", "C", "F")

fixed.assignTo(tower_points[0])
slide.assignTo(deck_points_backwards[-1])
slide.assignTo(deck_points_forwards[-1])

In [ ]:
''' Mesh Attributes'''
# Cables modelles with single bar element
cable_mesh_attr = db.createMeshLine("Bar Mesh").setNumber("BRS2", 1)
# Deck and tower modelled with beam elements
beam_mesh_attr = db.createMeshLine("Beam mesh 1m").setSize("BMI21", 1)

cable_mesh_attr.assignTo(back_stays)
cable_mesh_attr.assignTo(front_stays)
beam_mesh_attr.assignTo(deck_lines)
beam_mesh_attr.assignTo(tower_lines)

# Create the model mesh
db.updateMesh()

# Loads

In [ ]:
# Automatic gravity in loadcase 1
loadset = db.getLoadset("Loadcase 1").setName("Self Weight")
win32.CastTo(loadset, "IFLoadcase").addGravity(True)